In [4]:
import datetime
import math
import credit_curve_data
import itertools
import collections
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
from scipy.stats import norm
from finmarkets import CreditCurve,DiscountCurve, ForwardRateCurve, OvernightIndexSwap, generate_swap_dates,InterestRateSwap 
import data_for_libor
import ois_data_project
from ois_data_project import ois_quotes,pricing_date
from scipy.optimize import minimize

maturity=pricing_date+relativedelta(months=120)

In [5]:
n_of_days_libor=(date(2028, 4, 30)-date(2013, 10, 31)).days
n_of_months_libor=int(n_of_days_libor/365*12)

In [6]:
payment_dates= []
payment_dates = generate_swap_dates(data_for_libor.pricing_date,n_of_months_libor,6)
payment_dates

[datetime.date(2013, 10, 31),
 datetime.date(2014, 4, 30),
 datetime.date(2014, 10, 31),
 datetime.date(2015, 4, 30),
 datetime.date(2015, 10, 31),
 datetime.date(2016, 4, 30),
 datetime.date(2016, 10, 31),
 datetime.date(2017, 4, 30),
 datetime.date(2017, 10, 31),
 datetime.date(2018, 4, 30),
 datetime.date(2018, 10, 31),
 datetime.date(2019, 4, 30),
 datetime.date(2019, 10, 31),
 datetime.date(2020, 4, 30),
 datetime.date(2020, 10, 31),
 datetime.date(2021, 4, 30),
 datetime.date(2021, 10, 31),
 datetime.date(2022, 4, 30),
 datetime.date(2022, 10, 31),
 datetime.date(2023, 4, 30),
 datetime.date(2023, 10, 31),
 datetime.date(2024, 4, 30),
 datetime.date(2024, 10, 31),
 datetime.date(2025, 4, 30),
 datetime.date(2025, 10, 31),
 datetime.date(2026, 4, 30),
 datetime.date(2026, 10, 31),
 datetime.date(2027, 4, 30),
 datetime.date(2027, 10, 31),
 datetime.date(2028, 4, 30)]

In [7]:
curve=[]

for rate in data_for_libor.euribor_6m:     
         
        rate_curve =rate['rate']       # the rates of the libor_curve
        curve.append(rate_curve)
curve

[0.005,
 0.0058,
 0.0066,
 0.0074,
 0.0082,
 0.009,
 0.0098,
 0.0106,
 0.0114,
 0.0122,
 0.013,
 0.0138,
 0.0146,
 0.0154,
 0.0162,
 0.017,
 0.0178,
 0.0186,
 0.0194,
 0.0202,
 0.021,
 0.0218,
 0.0226,
 0.0234,
 0.0242,
 0.025,
 0.0258,
 0.0266,
 0.0274,
 0.0282]

In [8]:
libor_curve_project= ForwardRateCurve(payment_dates,curve)

In [9]:
pillar_dates = [data_for_libor.pricing_date]

swaps=[]

for quote in ois_data_project.ois_quotes:
    swap = OvernightIndexSwap(
        100e6,
        
        # payment dates
        generate_swap_dates(
            ois_data_project.pricing_date,
            quote['maturity']
        ),
        
        # the fixed rate
        quote['rate']
        
    )
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

n_df_vector = len(pillar_dates)

In [10]:
def objective_function(x):
    
    curve = DiscountCurve(
        
        # today date
        ois_data_project.pricing_date,
        
        # pillar dates
        pillar_dates,
        
        # pillar discount factors
        x
        
    )
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [11]:
x0 = [1.0 for i in range(n_df_vector)]
bounds = [(0.01, 100.0) for i in range(n_df_vector)]

# constrain the first pillar discount factor = 1
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [12]:
discount = []
discount = list(result.x)
discount

[1.0,
 0.9999116694687552,
 0.9998068656394881,
 0.9997061924564141,
 0.9996101470407041,
 0.9995011051264483,
 0.9993920042741734,
 0.9992819989228551,
 0.9991669935188687,
 0.9990300368964334,
 0.998894667953371,
 0.9987486256502633,
 0.9986022365627873,
 0.9980727386272038,
 0.9974861202861907,
 0.9967448393581466,
 0.9958318671514552,
 0.9895732948895709,
 0.9781408080931735,
 0.9621607527046473,
 0.9423126839666933,
 0.9197479369581603,
 0.8952093965299698,
 0.8692056177868052,
 0.8422119347612174,
 0.8152192175874029,
 0.7883481252357906,
 0.712079917509956,
 0.6100504918899721,
 0.5303022095774865,
 0.4645290818431267]

In [13]:
pillar_dates

[datetime.date(2013, 10, 31),
 datetime.date(2013, 11, 30),
 datetime.date(2013, 12, 31),
 datetime.date(2014, 1, 31),
 datetime.date(2014, 2, 28),
 datetime.date(2014, 3, 31),
 datetime.date(2014, 4, 30),
 datetime.date(2014, 5, 31),
 datetime.date(2014, 6, 30),
 datetime.date(2014, 7, 31),
 datetime.date(2014, 8, 31),
 datetime.date(2014, 9, 30),
 datetime.date(2014, 10, 31),
 datetime.date(2015, 1, 31),
 datetime.date(2015, 4, 30),
 datetime.date(2015, 7, 31),
 datetime.date(2015, 10, 31),
 datetime.date(2016, 10, 31),
 datetime.date(2017, 10, 31),
 datetime.date(2018, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2020, 10, 31),
 datetime.date(2021, 10, 31),
 datetime.date(2022, 10, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2024, 10, 31),
 datetime.date(2025, 10, 31),
 datetime.date(2028, 10, 31),
 datetime.date(2033, 10, 31),
 datetime.date(2038, 10, 31),
 datetime.date(2043, 10, 31)]

In [14]:
discount_curve_project = DiscountCurve(pricing_date, pillar_dates, discount)

In [15]:
def expon_icdf(p, lambd=1):
    #Inverse CDF of exponential distribution - i.e. the quantile function.
    return -np.log(1-p)/lambd

In [16]:
#characteristics of the swap:

credit_var_date = date(2014, 1, 31)
notional=100e6
frequency_floating_leg=6
maturity_in_n_months=120 
sigma=0.02 
random_fixed_rate=0.0002708 # a random paramater


In [17]:
#Compute the value of the Interest Rate Swap

irs_random=InterestRateSwap(pricing_date,notional,random_fixed_rate,frequency_floating_leg,maturity_in_n_months) 
                                                        
Npv_irs_random=irs_random.npv(discount_curve_project,libor_curve_project)

In [18]:
swap_rate=irs_random.swap_rate(discount_curve_project,libor_curve_project)
#  my fixed_rate, i.e. the value that makes the NPV of the IRS=0
swap_rate

0.026056559180472356

In [19]:
irs_fair=InterestRateSwap(pricing_date,notional,swap_rate,frequency_floating_leg,maturity_in_n_months)

Npv_irs_fair=irs_fair.npv(discount_curve_project,libor_curve_project)
Npv_irs_fair # if Npv_irs_fair=0 -> my fixed rate is corret

0.0

In [20]:
A=irs_fair.annuity(discount_curve_project)
S=irs_fair.swap_rate(discount_curve_project,libor_curve_project)

In [21]:
#credit curve data contains both pillar dates and rate dates, i.e. the non survival probability for each pillar date
pillar_dates = []
rate_dates=[]

for quote in credit_curve_data.survival_probabilities:  
        pillar_dates.append(quote['date']) 
for rate in credit_curve_data.survival_probabilities:
        rate_dates.append(rate['ndp'])



In [22]:
cc = CreditCurve(
    pillar_dates,
    rate_dates
)

In [23]:
#x=pillar_dates
#y = rate_dates

# Create a trace
#trace = go.Scatter(
    #x = pillar_dates,
    #y = rate_dates
#)

#data = [trace]

#py.iplot(data, filename='basic-line')

In [24]:
cc.ndp(pricing_date) 
#my assumption: the reference entity can't default on the pricing date

1.0

In [25]:
cc.ndp(date(2033, 12, 20))# the last NDP of the dataset

0.539046016487758

In [26]:
numdays=(maturity-pricing_date).days

numdays 

# the number of days between final date and pricing date

3652

In [27]:
date_list = [pricing_date + datetime.timedelta(days=x) for x in range(0, numdays)] 

In [28]:
n_of_days=[(x-pricing_date).days for x in date_list]

In [29]:
epsilon=[]

d=date_list[0]

while d<= date_list[-1]:
    
    hazard_rate=cc.hazard(d)
    epsilon.append(hazard_rate/365) # divided by 365 to obtain a daily hazard rate 
    d+=relativedelta(days=1)

In [30]:
#x=date_list
#y = epsilon

# Create a trace
#trace = go.Scatter(
    #x = x,
    #y = epsilon
#)

#data = [trace]

#py.iplot(data, filename='basic-line')

In [31]:
hazard_function=[]        
for i in range(3651):
    c=sum(epsilon[:i])
    hazard_function.append(c)
hazard_function

[0,
 6.803421524814723e-05,
 0.0001360684304963902,
 0.00020410264574445924,
 0.00027213686099266125,
 0.00034017107624088037,
 0.0004082052914890223,
 0.0004762395067372361,
 0.0005442737219853592,
 0.0006123079372334723,
 0.0006803421524816781,
 0.0007483763677298782,
 0.0008164105829779954,
 0.0008844447982261965,
 0.0009524790134743363,
 0.0010205132287225134,
 0.0010885474439707363,
 0.001156581659218813,
 0.0012246158744670173,
 0.0012926500897152001,
 0.0013606843049633445,
 0.0014287185202115666,
 0.0014967527354596702,
 0.0015647869507079252,
 0.001632821165956067,
 0.001700855381204186,
 0.0017688895964523943,
 0.0018369238117006026,
 0.0019049580269487434,
 0.0019729922421968813,
 0.002041026457445103,
 0.0021090606726931827,
 0.0021770948879413607,
 0.002245129103189565,
 0.0023131633184377462,
 0.0023811975336858744,
 0.0024492317489340533,
 0.0025172659641821858,
 0.0025853001794304185,
 0.0026533343946785856,
 0.0027213686099267654,
 0.002789402825174946,
 0.002857437040

In [32]:
values_hazard_function=[] # probability of default
i=0
for item in hazard_function:
    K = 1 - np.exp(-hazard_function[i])
    values_hazard_function.append(K)
    i+=1
values_hazard_function

[0.0,
 6.803190097337541e-05,
 0.0001360591736073724,
 0.00020408181821651716,
 0.000272099835115891,
 0.0003401132246203531,
 0.0004081219870445407,
 0.00047612612270342414,
 0.0005441256319115295,
 0.0006121205149838271,
 0.0006801107722349542,
 0.00074809640397977,
 0.0008160774105326896,
 0.0008840537922085723,
 0.0009520255493219443,
 0.0010199926821874428,
 0.001087955191119816,
 0.001155913076433257,
 0.0012238663384428472,
 0.0012918149774626686,
 0.0013597589938074695,
 0.001427698387791776,
 0.0014956331597297812,
 0.0015635633099363444,
 0.0016314888387254367,
 0.0016994097464116953,
 0.0017673260333095353,
 0.001835237699733372,
 0.0019031447459972872,
 0.0019710471724158074,
 0.0020389449793032366,
 0.002106838166973657,
 0.0021747267357415945,
 0.0022426106859211314,
 0.002310490017826461,
 0.002378364731771776,
 0.00244623482807127,
 0.0025141003070391355,
 0.002581961168989455,
 0.0026498174142363107,
 0.0027176690430937844,
 0.0027855160558760694,
 0.002853358452897025

In [33]:
#x=date_list
#y = values_hazard_function

# Create a trace
#trace = go.Scatter(
    #x = x,
    #y = values_hazard_function
#)

#data = [trace]

#py.iplot(data, filename='basic-line')

In [34]:
dictionary_with_dates=dict(zip(date_list,values_hazard_function))

In [35]:
dictionary_with_dates_sorted=collections.OrderedDict(sorted(dictionary_with_dates.items()))


In [36]:
percentiles_for_each_value_of_my_cdf=[]
list_values_cdf=[]
for v in values_hazard_function:
    list_values_cdf.append(v)
    inverse_cdf=expon_icdf(v)
    percentiles_for_each_value_of_my_cdf.append(inverse_cdf)
percentiles_for_each_value_of_my_cdf

[-0.0,
 6.803421524811404e-05,
 0.00013606843049639974,
 0.0002041026457445085,
 0.0002721368609926683,
 0.0003401710762409066,
 0.00040820529148904994,
 0.00047623950673727973,
 0.0005442737219853543,
 0.0006123079372334978,
 0.0006803421524816227,
 0.0007483763677298848,
 0.0008164105829780172,
 0.0008844447982262185,
 0.0009524790134743759,
 0.0010205132287225086,
 0.0010885474439707688,
 0.001156581659218774,
 0.0012246158744670527,
 0.0012926500897151544,
 0.0013606843049633166,
 0.0014287185202115763,
 0.001496752735459658,
 0.0015647869507079755,
 0.0016328211659560738,
 0.0017008553812041863,
 0.0017688895964523461,
 0.0018369238117006074,
 0.0019049580269487121,
 0.0019729922421968687,
 0.0020410264574450845,
 0.0021090606726931653,
 0.002177094887941384,
 0.0022451291031895903,
 0.002313163318437765,
 0.0023811975336859113,
 0.002449231748934054,
 0.0025172659641822396,
 0.0025853001794304237,
 0.002653334394678584,
 0.0027213686099267194,
 0.002789402825174961,
 0.0028574370

In [37]:
new_dictionary_3=dict(zip(date_list,percentiles_for_each_value_of_my_cdf))
new_dictionary_4=collections.OrderedDict(sorted(new_dictionary_3.items()))
new_dictionary_4

OrderedDict([(datetime.date(2013, 10, 31), -0.0),
             (datetime.date(2013, 11, 1), 6.803421524811404e-05),
             (datetime.date(2013, 11, 2), 0.00013606843049639974),
             (datetime.date(2013, 11, 3), 0.0002041026457445085),
             (datetime.date(2013, 11, 4), 0.0002721368609926683),
             (datetime.date(2013, 11, 5), 0.0003401710762409066),
             (datetime.date(2013, 11, 6), 0.00040820529148904994),
             (datetime.date(2013, 11, 7), 0.00047623950673727973),
             (datetime.date(2013, 11, 8), 0.0005442737219853543),
             (datetime.date(2013, 11, 9), 0.0006123079372334978),
             (datetime.date(2013, 11, 10), 0.0006803421524816227),
             (datetime.date(2013, 11, 11), 0.0007483763677298848),
             (datetime.date(2013, 11, 12), 0.0008164105829780172),
             (datetime.date(2013, 11, 13), 0.0008844447982262185),
             (datetime.date(2013, 11, 14), 0.0009524790134743759),
             (date

In [38]:
# creates a list of random numbers between 0 and 1, uniformly distributed with mean 0 and variance 1
list_of_random_numbers=[]
list_of_random_numbers=np.random.uniform(0,1,10000)
pc=list_of_random_numbers.tolist()
pc

[0.5292226154813668,
 0.07348411378330333,
 0.6006108035534674,
 0.9124795220099968,
 0.05857474886867775,
 0.6981101761759199,
 0.9532302843505062,
 0.010386892343131726,
 0.5596779930046687,
 0.38315464771006236,
 0.6894619098015053,
 0.35990797211574055,
 0.7816346493536175,
 0.6799456447242135,
 0.4401533439518739,
 0.21857422943607296,
 0.8380311132566762,
 0.589110339283699,
 0.6013071665033849,
 0.9512449464347004,
 0.20725929236143203,
 0.14906967461552978,
 0.3937503926981071,
 0.829369804753904,
 0.44342632148736727,
 0.6775585590735063,
 0.4020014366109619,
 0.9818297420009585,
 0.3812335884857432,
 0.9476493379390004,
 0.6168211846983682,
 0.29810046495976505,
 0.9039631988887922,
 0.6624949313065174,
 0.5022486125151082,
 0.3118779024854984,
 0.8597709426161753,
 0.1873152202846461,
 0.9765626477646775,
 0.36668719257350446,
 0.2527996552044296,
 0.2725553645981633,
 0.6167794064124169,
 0.7460664773810518,
 0.7568168802764866,
 0.8255571596353422,
 0.6843652326680076,
 0.

In [39]:
percentiles_of_the_random_numbers=[]
for item in pc:
    q=-1*np.log(1-item) 
    percentiles_of_the_random_numbers.append(q)
percentiles_of_the_random_numbers

[0.7533699410313723,
 0.07632408692050903,
 0.9178189078241207,
 2.435882478828828,
 0.060360327418246505,
 1.19769314994549,
 3.062519387025624,
 0.01044121258245541,
 0.8202489856495119,
 0.48313693108016437,
 1.1694487113852476,
 0.4461433193965684,
 1.5215856985842973,
 1.1392644373761485,
 0.5800923612362965,
 0.24663511691532616,
 1.820351019335064,
 0.8894305659396518,
 0.9195639994325339,
 3.0209464240329647,
 0.23225908731189038,
 0.1614250275743901,
 0.5004634844801089,
 1.7682566651776859,
 0.585955720781032,
 1.1318337377561858,
 0.514166927393859,
 4.007969197498118,
 0.4800274417915781,
 2.9497906949260853,
 0.9592535180732542,
 0.3539649976761916,
 2.3430238160637726,
 1.0861747504312673,
 0.6976545485281217,
 0.37378898947666545,
 1.9644780691956805,
 0.20741196944639492,
 3.7534242798976214,
 0.45679081234749447,
 0.29142193061743615,
 0.3182173853728654,
 0.959144493234487,
 1.370682768191418,
 1.4139405403306498,
 1.7461581533801143,
 1.1531695334289784,
 1.375662906

In [40]:
giorni=[]
for item in percentiles_of_the_random_numbers:
    c=min(new_dictionary_4, key=lambda y:abs(float(new_dictionary_4[y])-item))
    giorni.append(c)
giorni 

[datetime.date(2023, 10, 29),
 datetime.date(2016, 7, 9),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2016, 1, 7),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2014, 4, 2),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2021, 11, 22),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2021, 6, 6),
 datetime.date(2019, 2, 23),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023, 10, 29),
 datetime.date(2023

In [42]:
default_days=[]
for item in giorni:
    if  item<=credit_var_date:
        default_days.append(item)

len(default_days)
print (default_days)

[datetime.date(2014, 1, 17), datetime.date(2013, 10, 31), datetime.date(2013, 11, 8), datetime.date(2014, 1, 9), datetime.date(2013, 11, 25), datetime.date(2014, 1, 9), datetime.date(2013, 12, 30), datetime.date(2013, 11, 13), datetime.date(2013, 11, 18), datetime.date(2014, 1, 12), datetime.date(2013, 12, 22), datetime.date(2013, 11, 20), datetime.date(2013, 11, 26), datetime.date(2014, 1, 10), datetime.date(2013, 12, 3), datetime.date(2014, 1, 23), datetime.date(2013, 11, 16), datetime.date(2013, 11, 27), datetime.date(2013, 11, 18), datetime.date(2013, 11, 15), datetime.date(2013, 12, 29), datetime.date(2014, 1, 17), datetime.date(2014, 1, 14), datetime.date(2013, 11, 28), datetime.date(2013, 11, 15), datetime.date(2013, 12, 21), datetime.date(2014, 1, 4), datetime.date(2014, 1, 18), datetime.date(2013, 12, 14), datetime.date(2013, 12, 4), datetime.date(2013, 11, 15), datetime.date(2013, 11, 15), datetime.date(2013, 11, 11), datetime.date(2013, 11, 23), datetime.date(2014, 1, 25), d

In [43]:
# finds the days on which defaults occurs more than once.
seen = set()
for n in default_days:
    if n in seen:
        print("duplicate:", n)
    else:
        seen.add(n)

duplicate: 2014-01-09
duplicate: 2013-11-18
duplicate: 2014-01-17
duplicate: 2013-11-15
duplicate: 2013-11-15
duplicate: 2013-11-15
duplicate: 2013-12-22
duplicate: 2013-11-17
duplicate: 2013-11-15
duplicate: 2013-11-20
duplicate: 2013-11-27
duplicate: 2013-12-14
duplicate: 2013-11-27
duplicate: 2013-12-29
duplicate: 2014-01-12
duplicate: 2013-12-29
duplicate: 2013-11-08
duplicate: 2014-01-04
duplicate: 2013-12-15


In [44]:
# returns a random sample from the standard Gaussian distribution
from numpy.random import normal

# defines the number of Monte Carlo scenarios
n_scenarios = 10000

payoffs = []
npv_mc_list=[]

for item in default_days:
# performs the Monte Carlo simulation - loop over each scenario
    T=(maturity-item).days/365
    for i_scenario in range(n_scenarios):
    
    # simulates the swap rate in this scenario
        S_simulated = S * math.exp(-0.5 * sigma * sigma * T + sigma * math.sqrt(T) * normal())
    
    # calculate the swap NPV in this scenario
        swap_npv = irs_fair.notional * (S_simulated - irs_fair.fixed_rate)*(A/discount_curve_project.df(item))  
    
    # adds the payoff of the swap, in this scenario, to the list
       
        payoffs.append(swap_npv)
    
# calculates the NPV of the swap by taking the average of the payoffs across all the scenarios
    npv_mc = np.mean(payoffs)
    npv_mc_list.append(npv_mc)

npv_mc_list

[-7431.675318985045,
 104314.63477760131,
 73060.87835242007,
 36977.705956913465,
 51755.909718489434,
 64749.372541866534,
 55715.87569478137,
 34796.71365556486,
 32077.53923375185,
 24764.825166153525,
 24623.042613373596,
 25105.826761351294,
 32434.288358183527,
 28160.948112151094,
 18140.99489377508,
 19970.906763942567,
 12443.82993833795,
 11582.952159343578,
 19192.92434594184,
 19976.445378209057,
 21635.592317449453,
 22072.22486336809,
 16831.65948772955,
 9404.795398433687,
 7651.995112016635,
 7149.762048682564,
 6771.453560783534,
 5599.966302360185,
 150.2361352786726,
 839.9495666000565,
 3364.4281162746784,
 -368.383457338568,
 394.2839233927569,
 3746.873966495067,
 12388.469270548554,
 15558.938617722348,
 12110.27014785854,
 11055.219162965694,
 12740.87252582603,
 11994.013128307686,
 14760.030245672993,
 16213.203190382525,
 19634.645821154736,
 20402.080982799147,
 17511.637486666303,
 17656.566031163675,
 19282.75172553831,
 20114.67569754132,
 20752.79934174

In [45]:
recovery=0.40
losses_distribution=[]

i=0
for item in default_days:
        if npv_mc_list[i]>0:
            c=(1-recovery)*npv_mc_list[i]  # the loss is (1-recovery)*max(0,npv_mc_list) if the counterparty defaults
        else:
            indicatrix=0
            c=0*indicatrix
        i+=1
        losses_distribution.append(c)
losses_distribution

[0,
 62588.78086656079,
 43836.52701145204,
 22186.623574148078,
 31053.54583109366,
 38849.62352511992,
 33429.52541686882,
 20878.028193338916,
 19246.52354025111,
 14858.895099692114,
 14773.825568024156,
 15063.496056810776,
 19460.573014910115,
 16896.568867290654,
 10884.596936265048,
 11982.54405836554,
 7466.29796300277,
 6949.771295606147,
 11515.754607565103,
 11985.867226925433,
 12981.355390469671,
 13243.334918020853,
 10098.995692637729,
 5642.877239060213,
 4591.197067209981,
 4289.857229209538,
 4062.8721364701205,
 3359.979781416111,
 90.14168116720356,
 503.9697399600339,
 2018.656869764807,
 0,
 236.57035403565413,
 2248.1243798970404,
 7433.081562329132,
 9335.363170633409,
 7266.162088715124,
 6633.131497779416,
 7644.523515495617,
 7196.407876984612,
 8856.018147403796,
 9727.921914229515,
 11780.787492692842,
 12241.248589679488,
 10506.98249199978,
 10593.939618698205,
 11569.651035322986,
 12068.805418524791,
 12451.679605049092,
 11066.598626824407,
 9812.3607

In [ ]:
matricia=[]
for item in losses_distribution:
    if item>0:
        matricia.append(round(item,2))
        matricia.sort()

In [ ]:
x=date_list

In [ ]:
trace = go.Histogram(x=matricia, histnorm='probability',
                     xbins=dict(start=np.min(x),
                                size=0.05,
                                end=np.max(x)),
                     marker=dict(color='rgb(25, 25, 100)'))

layout = go.Layout(
    title="Loss Function"
)

fig = go.Figure(data=go.Data([trace]), layout=layout)
py.iplot(fig, filename='histogram-prob-dist')

In [ ]:
a = np.array(matricia)
p = np.percentile(a, 95) # returns the 95th percentile and thus the Credit Var
print (p)